In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt

from model import TPALSTM
from other_models import LSTM
import pandas as pd
from dataset import ElectricityDataModule

In [ ]:
data_df = np.load('data/kbest_dataset.npy')
data_df.shape

In [ ]:
data_splits = {
    "train": 0.7,
    "val": 0.15,
    "predict": 0.15
}

pred_horizon = 4

elec_dm = ElectricityDataModule(
    dataset_splits=data_splits,
    batch_size=256,
    window_size=24,
    pred_horizon=pred_horizon,
    data_style="kbest"
)

In [ ]:
run_name = f"{pred_horizon}ts-kbest"

**LSTM**

In [ ]:
wandb_logger_lstm = WandbLogger(
    name=f'{run_name}-LSTM',
    save_dir='logs',
    project='Time-Series project',
    log_model=True
)

checkpoint_loss_lstm = ModelCheckpoint(
    dirpath=f"checkpoints/{run_name}/LSTM",
    filename='BEST-{epoch}-{val_loss:.3f}-{val_score:.3f}',
    save_top_k=1,
    monitor="val/loss",
    mode="min"
)

lstm_trainer = pl.Trainer(
    max_epochs=80,
    # accelerator='gpu',
    callbacks=[checkpoint_loss_lstm],
    devices=1,
    logger=wandb_logger_lstm,
)

In [ ]:
lstm = LSTM(
    input_size=data_df.shape[1],
    lstm_hid_size=200,
    linear_hid_size=100,
    output_horizon=pred_horizon,
    n_layers=1,
    lr=1e-3
)

In [ ]:
lstm_trainer.fit(lstm, elec_dm)
wandb_logger_lstm.experiment.finish()

In [ ]:
elec_dm.setup("predict")

model_path = "checkpoints/4ts-kbest/LSTM/BEST-epoch=79-val_loss=0.000-val_score=0.000-v1.ckpt"
lstm = LSTM.load_from_checkpoint(model_path)

pred_dl = elec_dm.predict_dataloader()

y_pred = lstm_trainer.predict(lstm, pred_dl)

In [ ]:
batch_idx = 0
for i, batch in enumerate(pred_dl):
    inputs, labels = batch
    X, ytrue = inputs[batch_idx][:, -1], labels[batch_idx].squeeze()
    ypred = y_pred[i][batch_idx].squeeze()
    
    X = X.cpu().numpy()
    ytrue = ytrue.cpu().numpy()
    ypred = ypred.cpu().numpy()
    # print(inputs.shape)
    # print(labels.shape, y_pred[i].shape)

    plt.figure(figsize=(15, 5))
    plt.plot(range(0, 24), X, label="Input")
    plt.scatter(range(24, 24 + pred_horizon), ytrue, color='red', label="True price")
    plt.scatter(range(24, 24 + pred_horizon), ypred, color='green', label="Predicted price")
    plt.legend()
    plt.title("LSTM")
    plt.show()

    if i == 3:
        break

**Stacked LSTM**

In [ ]:
wandb_logger_stacked_lstm = WandbLogger(
    name=f'{run_name}-Stacked LSTM',
    save_dir='logs',
    project='Time-Series project',
    log_model=True
)

checkpoint_loss_stacked_lstm = ModelCheckpoint(
    dirpath=f"checkpoints/{run_name}/Stacked-LSTM",
    filename='BEST-{epoch}-{val_loss:.3f}-{val_score:.3f}',
    save_top_k=1,
    monitor="val/loss",
    mode="min"
)

stacked_lstm_trainer = pl.Trainer(
    max_epochs=80,
    accelerator='gpu',
    callbacks=[checkpoint_loss_stacked_lstm],
    strategy='auto',
    devices=1,
    logger=wandb_logger_stacked_lstm
)

In [ ]:
stacked_lstm = LSTM(
    input_size=data_df.shape[1],
    lstm_hid_size=200,
    linear_hid_size=100,
    output_horizon=pred_horizon,
    n_layers=2
)

In [ ]:
stacked_lstm_trainer.fit(stacked_lstm, elec_dm)
wandb_logger_stacked_lstm.experiment.finish()

In [ ]:
elec_dm.setup("predict")

model_path = "checkpoints/4ts-kbest/Stacked-LSTM/"
stacked_lstm = LSTM.load_from_checkpoint(model_path)

pred_dl = elec_dm.predict_dataloader()

y_pred = stacked_lstm_trainer.predict(stacked_lstm, pred_dl)

In [ ]:
batch_idx = 0
for i, batch in enumerate(pred_dl):
    inputs, labels = batch
    X, ytrue = inputs[batch_idx][:, -1], labels[batch_idx].squeeze()
    ypred = y_pred[i][batch_idx].squeeze()
    
    X = X.cpu().numpy()
    ytrue = ytrue.cpu().numpy()
    ypred = ypred.cpu().numpy()
    # print(inputs.shape)
    # print(labels.shape, y_pred[i].shape)

    plt.figure(figsize=(15, 5))
    plt.plot(range(0, 24), X, label="Input")
    plt.scatter(range(24, 24 + pred_horizon), ytrue, color='red', label="True price")
    plt.scatter(range(24, 24 + pred_horizon), ypred, color='green', label="Predicted price")
    plt.legend()
    plt.title("Stacked-LSTM")
    plt.show()

    if i == 3:
        break

**TPA-LSTM**

In [ ]:
wandb_logger_tpalstm = WandbLogger(
    name=f'{run_name}-TPA-LSTM',
    save_dir='logs',
    project='Time-Series project',
    log_model=True
)

checkpoint_loss_tpalstm = ModelCheckpoint(
    dirpath=f"checkpoints/{run_name}/TPA-LSTM",
    filename='BEST-{epoch}-{val_loss:.3f}-{val_score:.3f}',
    save_top_k=1,
    monitor="val/loss",
    mode="min"
)

tpalstm_trainer = pl.Trainer(
    max_epochs=80,
    # accelerator='gpu',
    callbacks=[checkpoint_loss_tpalstm]
    strategy='auto',
    devices=1,
    logger=wandb_logger_tpalstm
)

In [ ]:
tpa_lstm = TPALSTM(
    input_size=data_df.shape[1],
    hidden_size=50,
    output_horizon=pred_horizon,
    obs_len=24,
    n_layers=2
)

In [ ]:
tpalstm_trainer.fit(tpa_lstm, elec_dm)
wandb_logger_tpalstm.experiment.finish()

In [ ]:
elec_dm.setup("predict")

model_path = "checkpoints/4ts-kbest/TPA-LSTM/"
tpa_lstm = TPALSTM.load_from_checkpoint(model_path)

pred_dl = elec_dm.predict_dataloader()

y_pred = tpalstm_trainer.predict(tpa_lstm, pred_dl)

In [ ]:
batch_idx = 0
for i, batch in enumerate(pred_dl):
    inputs, labels = batch
    X, ytrue = inputs[batch_idx][:, -1], labels[batch_idx].squeeze()
    ypred = y_pred[i][batch_idx].squeeze()
    
    X = X.cpu().numpy()
    ytrue = ytrue.cpu().numpy()
    ypred = ypred.cpu().numpy()
    # print(inputs.shape)
    # print(labels.shape, y_pred[i].shape)

    plt.figure(figsize=(15, 5))
    plt.plot(range(0, 24), X, label="Input")
    plt.scatter(range(24, 24 + pred_horizon), ytrue, color='red', label="True price")
    plt.scatter(range(24, 24 + pred_horizon), ypred, color='green', label="Predicted price")
    plt.legend()
    plt.title("TPA-LSTM")
    plt.show()

    if i == 3:
        break